# Uncapacitated Multiple Allocation HLP Problem.
### Autor: Saúl Sosa Díaz
---
## El problema.
El Problema de Ubicación de Concentradores de Múltiple Asignación Sin Capacidad (UMAHLP), un tema vital en la investigación de operativa, involucra la determinación estratégica de la ubicación óptima de concentradores (hubs) y la asignación eficiente de nodos a estos puntos. En el UMAHLP, el objetivo principal es seleccionar concentradores de una lista de ubicaciones potenciales, asignando cada nodo no concentrador a uno o más de estos hubs. El fin es minimizar el costo total, que generalmente abarca el transporte entre nodos y concentradores, y en ocasiones, los costos operativos de los hubs. A diferencia de otros desafíos en la ubicación de concentradores, el UMAHLP se distingue por no tener restricciones de capacidad, permitiendo que cada concentrador maneje cualquier volumen de tráfico o demanda. Además, la característica de múltiple asignación permite que un nodo se asocie con más de un concentrador, incrementando la flexibilidad.

Consideremos un conjunto de nodos $ V$ y un número $ p$ de hubs por abrir en un grafo completo $ G=(V,A)$, donde $ |V|=n$. La cantidad de producto transportado del nodo $ i \in V$ al nodo $ j \in V$ se define como $ d_{ij}$. El costo unitario de transporte a lo largo del arco $ (i,j) \in A$ se representa por $ c_{ij}$.

Para modelar el costo de transporte, se introducen los siguientes factores, dependiendo del tipo de arco $ (i,j)$:

* Un factor de descuento $ \alpha$, donde $ 0 \le \alpha \le 1$, se aplica cuando el arco conecta dos hubs.
* Un factor de recolección $ \chi$, con $ \chi > \alpha$, se utiliza cuando el arco va de un nodo origen a un hub.

Tras establecer claramente el problema, estamos listos para abordar la construcción del modelo matemático.

**Variables**
 
 * $z_{ij}$ es 1 si el nodo $i\in V$ es asignado al hub $j\in V$ y 0 en otro caso. Si los índices coinciden de tal manera que $z_{jj}$, la variable toma el valor de 1 si se abre un hub en $j\in V$, y 0 en otro caso.
 * $X_{ijkm}$ es una variable continua que representa la fracción de la demanda $w_{ij}$ que se envía de $i$ a $j$ a través de los hubs $k$ y $m$. 
 * $g_{ijkm} = d_{ij}(\chi c_{ik} + \alpha c_{km} + \delta c_{mj} ) $, que representa el coste total del producto enviado desde $i$ a $j$ que pasa por las oficinas $k$ y $m$.

**Función Objetivo**
$$
\begin{align}
    \min & \sum_{k\in N} f_k z_{kk} + \sum_{i,j,k,m\in N} g_{ijkm} x_{ijkm}
\end{align}
$$

**Sujeto a**:
$$
\begin{array}{cc}
&\sum_{k\in N} z_{ik} = 1 & i\in N\\
& \sum_{k \in N} z_{kk} <= p \\
&z_{ik} \leq z_{kk} & i,k\in N\\
&z_{ik} \in \{0,1\} & i,k\in N\\
&\sum_{m\in N} x_{ijkm} = z_{ik} & i,j,k\in N\\
&\sum_{k\in N} x_{ijkm} = z_{jm} & i,j,m\in N\\
&x_{ijkm} \geq 0 & i,j,k,m\in N
\end{array}
$$


## Resolución del problema.
Importamos los paquetes necesarios.

In [1]:
import Pkg
Pkg.add("JuMP")
Pkg.add("GLPK")
Pkg.add("Plots")


using JuMP, GLPK, Random, Plots, Random, LinearAlgebra



   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


### Introducimos los datos.

In [18]:
Random.seed!(666) # Set seed for reproducibility

n = 5 # Random demand
V = 1:n 
w = rand(1:10, n, n) # Random demand
C = rand(1:10, n, n, n, n) # Random transportation costs
f = rand(1:10, n) # Fixed cost of opening a hub


5-element Vector{Int64}:
 4
 3
 1
 3
 5

Preparamos la representación de los datos.

In [19]:
function DrawInstance(client_location, factory_location, supply, demand)
    Xc = client_location[1]
    Yc = client_location[2]
    Xf = factory_location[1]
    Yf = factory_location[2]
    p = scatter( # Plot the clients
        Xc,
        Yc;
        label = nothing,
        markershape = :circle,
        markercolor = :green,
        markersize = 2 .* (2 .+ demand)
    )
    scatter!( # Plot the factories
        Xf,
        Yf;
        label = nothing,
        markershape = :star5,
        markercolor = :yellow,
        markersize = supply,
        markerstrokewidth = 2,
    )


    display(p)
end

function draw_solution(z, N, node_coordinates, scale_size = 1.0)
    # Extract X and Y coordinates of the nodes
    Xn = [node_coordinates[n][1] for n in N]
    Yn = [node_coordinates[n][2] for n in N]

    p = scatter( # Plot the nodes
        Xn,
        Yn;
        label = nothing,
        markershape = :circle,
        markercolor = :green,
        markersize = [scale_size for _ in N]
    )

    # Identify hubs and draw them
    hubs = []
    for k in N
        if value(z[k, k]) > 0.5
            push!(hubs, k)
            scatter!(
                [node_coordinates[k][1]],
                [node_coordinates[k][2]];
                label = "Hub $k",
                markershape = :star5,
                markercolor = :yellow,
                markersize = 2 * scale_size,
                markerstrokewidth = 2,
            )
        end
    end

    # Plot the connections between hubs
    for i in hubs
        for j in hubs
            if i != j
                plot!(
                    [node_coordinates[i][1], node_coordinates[j][1]],
                    [node_coordinates[i][2], node_coordinates[j][2]];
                    color = :red,
                    label = nothing,
                    linestyle = :dash,
                )
            end
        end
    end

    # Plot the connections between nodes and hubs
    for i in N
        for k in hubs
            if i != k && value(z[i, k]) > 0.5
                plot!(
                    [node_coordinates[i][1], node_coordinates[k][1]],
                    [node_coordinates[i][2], node_coordinates[k][2]];
                    color = :blue,
                    label = nothing,
                )
            end
        end
    end

    display(p)
end


draw_solution (generic function with 2 methods)

### Construir el modelo.

In [22]:
function solveCRPC()
    EPS = 0.00001
    model = Model(GLPK.Optimizer)

    # Variables
    @variable(model, y[k in V], Bin)
    @variable(model, X[i in V, j in V, k in V, m in V] >= 0)

    # Objetive
    @objective(model, Min, sum(w[i, j] * C[i, j, k, m] * X[i, j, k, m] for i in V, j in V, k in V, m in V) +
                        sum(f[k] * y[k] for k in V))

    # Constraints
    @constraint(model, [i in V, j in V], sum(X[i, j, k, m] for k in V, m in V) == 1)
    @constraint(model, [i in V, j in V, k in V, m in V], X[i, j, k, m] <= y[k])
    @constraint(model, [i in V, j in V, k in V, m in V], X[i, j, k, m] <= y[m])

    # Resolve model
    optimize!(model)

    # Show solutions
    if termination_status(model) == OPTIMAL
        println("Solution:")
        println("   Objective value: ", objective_value(model))
        # Display the hubs that are opened
        println("Hubs opened at the following locations:")
        for k in V
            if value(y[k]) > 0.5 # Considering y[k] as binary
                println(" - Hub at location $k")
            end
        end

        # Display the demand routes
        println("\nDemand routes:")
        for i in V, j in V
            if i != j
                println("Route from $i to $j:")
                for k in V, m in V
                    if value(X[i,j,k,m]) > 0
                        println("  - Fraction $(round(value(X[i,j,k,m]))) passes through hubs $k and $m")
                    end
                end
            end
        end

    else
        println("infeasible")
        return -1
    end
    
end


solveCRPC (generic function with 1 method)

Resolvemos y mostramos el resultado.

In [23]:
solveCRPC()

Solution:
   Objective value: 192.0
Hubs opened at the following locations:
 - Hub at location 1
 - Hub at location 2
 - Hub at location 3
 - Hub at location 4
 - Hub at location 5

Demand routes:
Route from 1 to 2:
  - Fraction 1.0 passes through hubs 2 and 4
Route from 1 to 3:
  - Fraction 1.0 passes through hubs 4 and 5
  - Fraction 0.0 passes through hubs 5 and 3
Route from 1 to 4:
  - Fraction 1.0 passes through hubs 1 and 3
  - Fraction 0.0 passes through hubs 1 and 4
Route from 1 to 5:
  - Fraction 1.0 passes through hubs 2 and 5
Route from 2 to 1:
  - Fraction 1.0 passes through hubs 1 and 4
  - Fraction 0.0 passes through hubs 2 and 2
Route from 2 to 3:
  - Fraction 1.0 passes through hubs 2 and 1
Route from 2 to 4:
  - Fraction 0.0 passes through hubs 3 and 1
  - Fraction 1.0 passes through hubs 3 and 2
Route from 2 to 5:
  - Fraction 0.0 passes through hubs 2 and 1
  - Fraction 1.0 passes through hubs 3 and 4
Route from 3 to 1:
  - Fraction 1.0 passes through hubs 1 and 1
  